<a href="https://colab.research.google.com/github/jasonjstewart/college-football/blob/master/Modeling_College_Football_FG_MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DATA CLEANING
## Data Imports
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import math

## Visualization imports
from matplotlib import pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

## Machine Learning Imports
import sklearn.metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

## For evaluating our Log Reg result
from sklearn import metrics

df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/college-football-data-fg-p5_final.csv')
# points_scored=[]
# for row in df['success_or_failure']:
#   if row==0:
#     points_scored.append(0)
#   else:
#     points_scored.append(3)
# df['points_scored']=points_scored
# df

# scoring=[]
# for row in df['scoring']:
#   if row==False:
#     scoring.append(0)
#   else:
#     scoring.append(1)
# df['scoring']=scoring
# df.describe()

# cf=[]
# for row in df['conference_game']:
#   if row==False:
#     cf.append(0)
#   else:
#     cf.append(1)
# df['conference_game']=cf
# df
# ns=[]
# for row in df['neutral_site']:
#   if row==False:
#     ns.append(0)
#   else:
#     ns.append(1)
# df['neutral_site']=ns

df.drop(['home','ppa','away','Unnamed: 0','Unnamed: 0.1','offense','id','defense','drive_result','yards_to_goal','play_text','kicker_name','defense_conference','attendance','next_drive_defense_conference','next_drive_offense_conference','drive_id','season','next_drive_id','next_drive_game_id','next_drive_start_time','next_drive_yard_line','next_drive_yards_to_goal','next_drive_end_time','next_drive_elapsed','next_drive_drive_result'], axis=1, inplace=True)
df.drop(['next_drive_scoring','next_drive_start_period','next_drive_start_yardline','next_drive_start_yards_to_goal','next_drive_end_period','next_drive_end_yardline','next_drive_end_yards_to_goal','next_drive_plays','next_drive_yards','next_drive_points_scored'], axis=1, inplace=True)
#df.drop(['less_than_20_percentage','between_20-29_percentage','between_30-39_percentage','between_40-49_percentage','50_plus_percentage'], axis=1, inplace=True)
df.drop(['play_type','down','fg_result','fg_percentage'], axis=1, inplace=True)
#'offense_conference'
df.dropna(axis=1)
df = df.reindex(sorted(df.columns), axis=1)
df.head(3)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,50_plus_percentage,between_20-29_percentage,between_30-39_percentage,between_40-49_percentage,clock,defense_score,distance,drive_start_yard_line,fg_length,fg_points,less_than_20_percentage,number_attempts_50_plus,number_of_plays_in_drive,offense_conference,offense_score,period,total_attempts,yards_gained_during_drive
0,0.285714,1.000000,0.851064,0.533333,746,7,5,37,36,3,1.0,7,7,American Athletic,3,2,115,18
1,0.500000,0.909091,0.730769,0.560000,380,3,5,77,22,3,0.0,4,15,Big Ten,10,2,78,72
2,0.000000,1.000000,0.681818,0.100000,698,0,5,58,23,3,1.0,1,10,Big 12,3,1,74,52


In [ ]:
#FIX SKEWNESS, NEED TO ACCOUNT FOR NEGATIVE SKEWNESS
#MAY NEED TO LOOK AT https://app.myeducator.com/reader/web/1702a/p1/wb00d/ in the textbook, 
# not sure if you have access to this one or this one https://app.myeducator.com/reader/web/1702/p1/re4ji/
#Look into binning if needed, probably wont.


def fix_skew(df, feature):
  import numpy as np
  # check for positive skewness
  if df[feature].skew() > 1:
    # generate the new columns to test
    df[feature + '_SR'] = np.sqrt(df[feature])
    df[feature + '_CR'] = np.cbrt(df[feature])
    df[feature + '_LN'] = np.log(df[feature] + 1)
    # df[feature + '_BIN'] = bin_numeric(df[feature])[2]
    print(df.skew())
    
    # get dummies
    for col in df:
      if not pd.api.types.is_numeric_dtype(df[col]):
        df = df.join(pd.get_dummies(df[col], prefix=col, drop_first=True))

    # get numeric columns
    df = df.select_dtypes(np.number)
    df_minmax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns=df.columns)

    # test original feature
    y = df_minmax[feature]
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test square root feature
    y = df_minmax[feature + '_SR']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test cube root feature
    y = df_minmax[feature + '_CR']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test log feature
    y = df_minmax[feature + '_LN']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test binning feature
    # y = df_minmax[feature + '_BIN']
    # X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR', feature + '_BIN']).assign(const=1)

    # results = sm.OLS(y, X).fit()
    # print(results.summary())


  elif df[feature].skew() < -1:
    # here we would fix negatively skewed variables if there were any in this dataset
    #$*$*$*$**$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$%^~!$@$$&^^%$@$!#@#&#$@!##$#*&#^@%%%#$!@$%$#&%^*@#$%!@#$%@#$^#$&#$@#$!#$@^#%&#$&*$%^*$%@#^$@#$%#$&$%*#$%#@&$^#$%&#%^*
    #need to account for negative skewness
    return
  else:
    # return if skewness is between -1 and 1; negligible
    return

In [ ]:
#CORRECT OUTLIERS, this first part checks to see if there are outliers
outliers = pd.DataFrame(columns=['min', 'count below', 'max', 'count above'])

# save the max to compare later
# peak_max = df[''].mean() + (df[''].std() * 3)

# Loop through each column in the dataframe except those that are: 1) categorical, 2) empty, or 3) binary (0/1)
for col in df:
  if pd.api.types.is_numeric_dtype(df[col]) and (len(df[col].value_counts()) > 0) and not all(df[col].value_counts().index.isin([0, 1])):
    # Calculate the theoretical min/max
    min = int(df[col].mean() - (df[col].std() * 3))
    max = int(df[col].mean() + (df[col].std() * 3))
    outliers.loc[col] = (min, df[col][df[col] < min].count(), max, df[col][df[col] > max].count())

outliers

#Goes through each column in DF to replace outliers with min or max
def manage_outliers(df):
  for col in df:
    if pd.api.types.is_numeric_dtype(df[col]) and (len(df[col].value_counts()) > 0) and not all(df[col].value_counts().index.isin([0, 1])):
      min = df[col].mean() - (df[col].std() * 3)
      max = df[col].mean() + (df[col].std() * 3)

      # Once the min/max is calculated for a column, take an inner loop through the row values of each column to replace values
      for i, value in enumerate(df[col]): 
        if value < min:      
          df.at[i, col + '_3std'] = min  # replace with min
          df.at[i, 'outlier'] = 1         # identify the record as an outlier for future filtering
        elif value > max:    
          df.at[i, col + '_3std'] = max   # replace with max
          df.at[i, 'outlier'] = 1
        else:                
          df.at[i, col + '_3std'] = value # use original value
          df.at[i, 'outlier'] = 0
    
  # I don't like binary variables like "outlier" becoming floats; cast it back to an int
  # if 'outlier' in df.columns:
  #   df = df.astype({"outlier": int})

  # View the cleaned dataset:
  print(df.head())

In [ ]:
#There shouldnt be any missing data, like I am 99% sure there isnt missing data but could be good to double check

#THIS BELOW COMES STRAIGHT FROM THE BOOK BUT IT SHOULD WORK
def manage_missing_data(df):
      import pandas as pd
      from scipy import stats

      for col_y in df:                                        # for each column in df to fill
        if df[col_y].isnull().sum() > 0:                      # if there is missing data to fill
          if is_numeric(df[col_y]):                           # and if that column is numeric
            r_list = df.corr()[col_y].abs()                   # create a correlation matrix for just the col_y feature; get absolute values to eliminate sign (+/-)
            r_list.sort_values(inplace=True, ascending=False) # sort the list descending (we want the second item in the list because the first is itself where r = 1.0)

            col_x = r_list.index[1] # Store the index name of the highest correlation

            # Create a new df with missing values of y and x dropped
            clean_df = df.copy()
            clean_df.dropna(subset=[col_y, col_x], inplace=True)

            # Calculate the regression line: y = m(x) + b
            m, b, r, p, err = stats.linregress(clean_df[col_x], clean_df[col_y])

            # enumerate() allows us to include a count-based index (i) along with the value
            for i, value in enumerate(df[col_y].values):
              if not pd.notnull(value):
                imputed_value = round((m * df[col_x][i]) + b, 1)

                # if the prediction is outside the range of actual values, then replace it with the min/max
                if imputed_value < df[col_y].min():
                  df[col_y][i] = df[col_y].min()
                elif imputed_value > df[col_y].max():
                  df[col_y][i] = df[col_y].max()
                else:
                  df[col_y][i] = imputed_value
      return df

In [ ]:
def data_cleaning(df):
  manage_missing_data(df)
  manage_outliers(df)
  for col in df:
    fix_skew(df, col)
df.to_csv('college-football-data-p5-4th-down.csv') #change the name and it will output
df.head(3)

,50_plus_percentage,between_20-29_percentage,between_30-39_percentage,between_40-49_percentage,clock,defense_score,distance,drive_start_yard_line,fg_length,fg_points,less_than_20_percentage,number_attempts_50_plus,number_of_plays_in_drive,offense_conference,offense_score,period,total_attempts,yards_gained_during_drive
0,0.285714,1.000000,0.851064,0.533333,746,7,5,37,36,3,1.0,7,7,American Athletic,3,2,115,18
1,0.500000,0.909091,0.730769,0.560000,380,3,5,77,22,3,0.0,4,15,Big Ten,10,2,78,72
2,0.000000,1.000000,0.681818,0.100000,698,0,5,58,23,3,1.0,1,10,Big 12,3,1,74,52


In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 
# df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/college-football-data-p5-4th-down-final.csv')
#df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/college-football-data-fg-p5_final.csv')
df = df.dropna(how='any',axis=0) 

pd.isnull(df).sum()

50_plus_percentage           0
between_20-29_percentage     0
between_30-39_percentage     0
between_40-49_percentage     0
clock                        0
defense_score                0
distance                     0
drive_start_yard_line        0
fg_length                    0
fg_points                    0
less_than_20_percentage      0
number_attempts_50_plus      0
number_of_plays_in_drive     0
offense_conference           0
offense_score                0
period                       0
total_attempts               0
yards_gained_during_drive    0
dtype: int64

In [ ]:
def mlr_prepare(df):
  import numpy as np
  import pandas as pd
  from sklearn import preprocessing

  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      print(col)
      df = df.join(pd.get_dummies(df[col], prefix=col, drop_first=False))
      df.drop([col], axis=1, inplace=True)

  return df
df = mlr_prepare(df)
df

offense_conference


,50_plus_percentage,between_20-29_percentage,between_30-39_percentage,between_40-49_percentage,clock,defense_score,distance,drive_start_yard_line,fg_length,fg_points,less_than_20_percentage,number_attempts_50_plus,number_of_plays_in_drive,offense_score,period,total_attempts,yards_gained_during_drive,offense_conference_ACC,offense_conference_American Athletic,offense_conference_Big 12,offense_conference_Big East,offense_conference_Big Ten,offense_conference_Conference USA,offense_conference_FBS Independents,offense_conference_Mid-American,offense_conference_Mountain West,offense_conference_None,offense_conference_Pac-10,offense_conference_Pac-12,offense_conference_SEC,offense_conference_Sun Belt,offense_conference_Western Athletic
0,0.285714,1.000000,0.851064,0.533333,746,7,5,37,36,3,1.0,7,7,3,2,115,18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.500000,0.909091,0.730769,0.560000,380,3,5,77,22,3,0.0,4,15,10,2,78,72,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.000000,1.000000,0.681818,0.100000,698,0,5,58,23,3,1.0,1,10,3,1,74,52,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0.000000,1.000000,0.666667,0.241379,560,0,8,97,26,3,1.0,5,11,3,1,98,89,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.000000,1.000000,0.666667,0.241379,395,0,6,43,35,3,1.0,5,7,6,1,98,25,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27283,1.000000,1.000000,1.000000,1.000000,473,13,3,69,24,3,1.0,2,11,13,3,34,63,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
27284,1.000000,1.000000,1.000000,1.000000,631,13,7,50,24,3,1.0,2,10,23,4,34,43,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
27285,1.000000,1.000000,0.625000,1.000000,205,23,3,90,27,3,1.0,2,5,23,4,34,80,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
27286,1.000000,0.846154,0.500000,0.500000,645,17,6,75,24,3,0.0,4,7,48,4,41,69,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

label = "fg_points"

# Set label and features
y = df[label]                    
X = df.select_dtypes(np.number).assign(const=1)
X = X.drop(columns=[label])

# Run the multiple linear regression model
model = sm.OLS(y, X)
results = model.fit()

# View results
print(df.skew())
print(results.summary())

50_plus_percentage                       0.651632
between_20-29_percentage                -3.070506
between_30-39_percentage                -1.738785
between_40-49_percentage                -0.753577
clock                                    0.157599
defense_score                            1.139443
distance                                 1.408784
drive_start_yard_line                   -0.764803
fg_length                                0.161839
fg_points                               -0.974684
less_than_20_percentage                  0.045904
number_attempts_50_plus                  1.618918
number_of_plays_in_drive                 1.747225
offense_score                            0.992549
period                                   0.325244
total_attempts                           0.493998
yards_gained_during_drive               -0.436520
offense_conference_ACC                   1.663956
offense_conference_American Athletic    10.058503
offense_conference_Big 12                1.916244


In [ ]:
# def CorrelationTable(df, width, height):

#     # Create Correlation df from source df
#     corr = df.corr()
#     # Plot figsize
#     fig, ax = plt.subplots(figsize=(width, height))
#     # Drop self-correlations
#     dropSelf = np.zeros_like(corr)
#     dropSelf[np.triu_indices_from(dropSelf)] = True 

#     # Generate Heat Map, allow annotations and place floats in map
#     sns.heatmap(corr, cmap="RdBu", annot=True, fmt=".2f", mask=dropSelf, 
#         xticklabels=corr.columns, 
#             yticklabels=corr.columns, ax=ax, linewidths=.5, cbar_kws={"shrink": .7},
#             vmin = -1, vmax=1, center=0)
#     plt.title('Correlation HeatMap',fontsize=14)
#     plt.show() 
# #Calling the function
# CorrelationTable(df,24,20) 

In [ ]:

x_vars = df.drop(columns = ['fg_points'])

y_vars = df.fg_points

In [ ]:
# Import this function
from sklearn.model_selection import train_test_split

# Create partitions
x_train, x_valid, y_train, y_valid = train_test_split(x_vars,y_vars, test_size=0.4, random_state=1)

In [ ]:
print('x_train contains:',len(x_train) ,'rows')
print('y_train contains:',len(y_train) ,'rows')
print(' x_valid contains:',len(x_valid) ,'rows')
print(' y_valid contains:',len(y_valid) ,'rows')

x_train contains: 16372 rows
y_train contains: 16372 rows
 x_valid contains: 10916 rows
 y_valid contains: 10916 rows


In [ ]:
x_train = x_train.sort_index()
y_train = y_train.sort_index()
x_valid = x_valid.sort_index()
y_valid = y_valid.sort_index()
x_train

,50_plus_percentage,between_20-29_percentage,between_30-39_percentage,between_40-49_percentage,clock,defense_score,distance,drive_start_yard_line,fg_length,less_than_20_percentage,number_attempts_50_plus,number_of_plays_in_drive,offense_score,period,total_attempts,yards_gained_during_drive,offense_conference_ACC,offense_conference_American Athletic,offense_conference_Big 12,offense_conference_Big East,offense_conference_Big Ten,offense_conference_Conference USA,offense_conference_FBS Independents,offense_conference_Mid-American,offense_conference_Mountain West,offense_conference_None,offense_conference_Pac-10,offense_conference_Pac-12,offense_conference_SEC,offense_conference_Sun Belt,offense_conference_Western Athletic
1,0.500000,0.909091,0.730769,0.560000,380,3,5,77,22,0.0,4,15,10,2,78,72,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0.000000,1.000000,0.666667,0.241379,560,0,8,97,26,1.0,5,11,3,1,98,89,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0.285714,1.000000,0.851064,0.533333,210,7,2,75,19,1.0,7,13,3,1,115,73,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0.285714,1.000000,0.851064,0.533333,48,14,11,66,47,1.0,7,11,10,2,115,36,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0.750000,1.000000,0.636364,0.583333,718,7,4,65,21,1.0,4,10,3,1,56,61,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27283,1.000000,1.000000,1.000000,1.000000,473,13,3,69,24,1.0,2,11,13,3,34,63,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
27284,1.000000,1.000000,1.000000,1.000000,631,13,7,50,24,1.0,2,10,23,4,34,43,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
27285,1.000000,1.000000,0.625000,1.000000,205,23,3,90,27,1.0,2,5,23,4,34,80,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
27286,1.000000,0.846154,0.500000,0.500000,645,17,6,75,24,0.0,4,7,48,4,41,69,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Now the partitions are ready, lets get ready to train the model

# Get  linear regression method from library
from sklearn.linear_model import LinearRegression

# Create multiple linear regression object
mlr = LinearRegression()

## step 2: fit (train) the linear model with the training data)

mlr.fit(x_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print("Intercept: {0:.3f} ".format(mlr.intercept_))

coeff_df = pd.DataFrame(mlr.coef_, x_train.columns, columns=['Coefficient'])
coeff_df

Intercept: 2.377 


,Coefficient
50_plus_percentage,0.197278
between_20-29_percentage,0.458365
between_30-39_percentage,0.685929
between_40-49_percentage,0.701515
clock,0.000195
defense_score,0.002045
distance,0.008431
drive_start_yard_line,-0.004905
fg_length,-0.044656
less_than_20_percentage,-0.016912


In [ ]:
# The model is trained. But the Linear Regression Model from the sklearn library
# does not included the ability to conveniently show the p-value of the 
# regression coefficients. Fortunately, the statsmodel library does this easily: 

# Get the P-Value of the coefficients
X2 = sm.add_constant(x_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())
#print(round(est2.pvalues.astype('float'),3))

                            OLS Regression Results                            
Dep. Variable:              fg_points   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     129.6
Date:                Tue, 16 Jun 2020   Prob (F-statistic):               0.00
Time:                        22:35:59   Log-Likelihood:                -26358.
No. Observations:               16372   AIC:                         5.278e+04
Df Residuals:                   16341   BIC:                         5.302e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [ ]:
y_pred = mlr.predict(x_valid)  
y_pred

array([2.13289246, 2.3657638 , 1.951834  , ..., 0.96140751, 1.95460935,
       2.88982995])

In [ ]:
## Create a dataframe to display the results of the actual and the predicted values

df = DataFrame({'Actual': y_valid, 'Predicted': y_pred})  
df.to_csv('results.csv')

## The actual and predicted values are pretty close for most records.

In [ ]:
# To check for the quality of our regression, use model quality metrics. 
# These metrics are calculated by comparing y_test (actual) and y_test (predicted).
    
R2 = sklearn.metrics.r2_score(y_valid, y_pred)
MAE = sklearn.metrics.mean_absolute_error(y_valid, y_pred)
RMSE = np.sqrt(sklearn.metrics.mean_squared_error(y_valid, y_pred))

print("R2: {0:.3f} ".format(R2))
print("MAE: {0:.1f} ".format(MAE))
print("RMSE: {0:.1f} ".format(RMSE))

#These results on the test partition mean our regression is pretty good

R2: 0.197 
MAE: 1.0 
RMSE: 1.2 


In [ ]:
x_train.head()

,50_plus_percentage,between_20-29_percentage,between_30-39_percentage,between_40-49_percentage,clock,defense_score,distance,drive_start_yard_line,fg_length,less_than_20_percentage,number_attempts_50_plus,number_of_plays_in_drive,offense_score,period,total_attempts,yards_gained_during_drive,offense_conference_ACC,offense_conference_American Athletic,offense_conference_Big 12,offense_conference_Big East,offense_conference_Big Ten,offense_conference_Conference USA,offense_conference_FBS Independents,offense_conference_Mid-American,offense_conference_Mountain West,offense_conference_None,offense_conference_Pac-10,offense_conference_Pac-12,offense_conference_SEC,offense_conference_Sun Belt,offense_conference_Western Athletic
1,0.500000,0.909091,0.730769,0.560000,380,3,5,77,22,0.0,4,15,10,2,78,72,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0.000000,1.000000,0.666667,0.241379,560,0,8,97,26,1.0,5,11,3,1,98,89,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0.285714,1.000000,0.851064,0.533333,210,7,2,75,19,1.0,7,13,3,1,115,73,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0.285714,1.000000,0.851064,0.533333,48,14,11,66,47,1.0,7,11,10,2,115,36,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0.750000,1.000000,0.636364,0.583333,718,7,4,65,21,1.0,4,10,3,1,56,61,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
